In [10]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# username and password

username = "aacuser"
password = "Aacuserpass"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open('Grazioso Salvare Logo.png', 'rb').read())

app.layout = html.Div([
    # Clickable logo image
    html.Center([
    html.A(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
        href="https://www.snhu.edu"
    ),
    html.B(html.H1('SNHU CS-340 Dashboard - Nawras Janoudi'))
    ]),
    
    html.Hr(),
    
    # Interactive filtering options
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset', # Default value
            labelStyle={'display':'inline-block'}
        )
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # Features for interactive data table
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################


# filter interactive data table with MongoDB queries
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
        
        # If a filter is selected, build the appropriate query
        if filter_type == 'Water':
            query = {
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte": 26, "$lte":  156}
            }
        elif filter_type == "Mountain":
            query = {
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte": 26, "$lte":  156}
            }
        elif filter_type == "Disaster":
            query = {
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte": 20, "$lte":  300}
            }
        else:
            query = {}
        
        # Execute query and return results
        results = db.read(query)
        
        # Convert results to DataFrame
        results_df = pd.DataFrame.from_records(results)
        
        # drop the '_id' column (if exists)
        if '_id' in results_df.columns:
            results_df.drop(columns=['_id'], inplace=True)
        
        # Convert back to a list for data table
        return results_df.to_dict('results')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
        if viewData is None:
            return []
        
        dff = pd.DataFrame.from_dict(viewData)
        
        # Check if the dataframe is empty
        if dff.empty:
            return [html.H4("No data to display.")]
        
        
        ####LOGIC TO DETERMINE USING PIE OR BAR CHART#####
        # Threshold for when to switch from a pie to bar chart
        CATEGORY_THRESHOLD = 10
        
        breed_counts = dff['breed'].value_counts()
        
        if len(breed_counts) <= CATEGORY_THRESHOLD: # Create Pie chart if under or equal 10 breeds
            figure = px.pie(
                breed_counts,
                names=breed_counts.index,
                values=breed_counts.values,
                title='Breed Distribution'
            )
        else:
            top_breeds = breed_counts.head(15)
            figure = px.pie(
                top_breeds,
                names=top_breeds.index,
                values=top_breeds.values,
                title='Breed Distribution'
            )
            
        return [
            dcc.Graph(figure=figure)
        ]
        
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if viewData is None:
        return 

    dff = pd.DataFrame.from_dict(viewData)

    # Check if the DataFrame is empty.
    if dff.empty:
        # Return a default, empty map if there are no results to display
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                   center=[30.26, -97.74], zoom=10, children=[
                dl.TileLayer(id="base-layer-id")
            ])
        ]

    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None or not index:
        row = 0
    else: 
        row = index[0]

    # Austin TX is at [30.26, -97.74]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.26, -97.74], zoom=9, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Connection Successful
Dash app running on http://127.0.0.1:17568/


/tmp/ipykernel_88805/1719015306.py:170: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

/tmp/ipykernel_88805/1719015306.py:170: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

/tmp/ipykernel_88805/1719015306.py:170: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

/tmp/ipykernel_88805/1719015306.py:170: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence